In [9]:
#import packages
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.blocking import BlockingScheduler
import numpy as np
import pickle
import pandas as pd
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
import category_encoders as cat_enc
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pickle
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

#util function
def preprocess_features(X):
    """Add any required feature preprocessing here, if it's not handled by the pickled model"""

    # Preprocess FlightDate
    X['FlightDate'] = pd.to_datetime(X['FlightDate'],format='%Y-%m-%d')
    X['Day_of_Week'] = X['FlightDate'].dt.day_name()
    X['day'] = X['FlightDate'].dt.day
    X['month'] = X['FlightDate'].dt.month
    X['year'] = X['FlightDate'].dt.year
    X = X.drop('FlightDate', axis=1)
    #X = X.drop('DepTime', axis=1)
    return X


#import model
model = pickle.load(open('customf.pkl', 'rb'))


#############################################
#############################################
#
#	Read SnowFlake Table
#
#############################################
#############################################

#import snowflake
import snowflake.connector
import pyarrow
import pandas

SNOW_USERNAME = 'DATAROBOT'
SNOW_PASSWORD = 'D@t@robot'
SNOW_SCHEMA   = 'FSDS'
SNOW_DATABASE = 'TEST_DB'
SNOW_WAREHOUSE = 'DEMO_WH'
SNOW_ACCOUNT  = 'PUBLIC'
TABLE = 'TEST_DB.FSDS.FSDS_data'
URL = 'datarobot_partner'

# Create Connection
ctx = snowflake.connector.connect(
                user = SNOW_USERNAME,
                password = SNOW_PASSWORD,
                account = URL,
                warehouse = SNOW_WAREHOUSE,
                database = SNOW_DATABASE,
                schema = SNOW_SCHEMA
                )

# Create a Cursor
cur = ctx.cursor()


#############################################
#############################################
#
#	Write data to table
#
#############################################
#############################################

import pandas
from snowflake.connector.pandas_tools import write_pandas

SNOW_USERNAME = 'DATAROBOT'
SNOW_PASSWORD = 'D@t@robot'
SNOW_SCHEMA   = 'FSDS'
SNOW_DATABASE = 'TEST_DB'
SNOW_WAREHOUSE = 'DEMO_WH'
SNOW_ACCOUNT  = 'PUBLIC'
TABLE = 'TEST_DB.FSDS.FSDS_predictions'
URL = 'datarobot_partner'

# Create the connection to the Snowflake database.
cnx = snowflake.connector.connect(
                user = SNOW_USERNAME,
                password = SNOW_PASSWORD,
                account = URL,
                warehouse = SNOW_WAREHOUSE,
                database = SNOW_DATABASE,
                schema = SNOW_SCHEMA
                )

In [13]:
import time

from timeloop import Timeloop
from datetime import timedelta

tl = Timeloop()

@tl.job(interval=timedelta(seconds=5000))
def sf_wb():
    # SQL - IMPORT DATA FROM SNOWFLAKE
    sql = 'SELECT * FROM "TEST_DB"."FSDS"."FSDS_data"'
    cur.execute(sql)
    # Fetch results into dataframe
    df = cur.fetch_pandas_all()
    # subset table
    df['Upload_Date'] = df['Upload_Date'].astype(str)
    df2 = df.loc[df["Upload_Date"] == "2020-01-10"]  # update this to current date to run it every day
    print("data imported")
    df2.rename(columns={'FLIGHTDATE': 'FlightDate'}, inplace=True)
    df2.rename(columns={'UNIQUECARRIER': 'UniqueCarrier'}, inplace=True)
    df2.rename(columns={'ORIGIN': 'Origin'}, inplace=True)
    df2.rename(columns={'DEST': 'Dest'}, inplace=True)
    df2.rename(columns={'DISTANCE': 'Distance'}, inplace=True)
    final_features = df2[['FlightDate', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']]
    final_features = preprocess_features(final_features)
    print("features generated")
    print(df2.dtypes)
    final_features['Distance'] = final_features['Distance'].astype(float)
    final_features['Distance'].values[final_features['Distance'] > 50000000000] = 50000000000 / 10  # fix anomaly values
    model = pickle.load(open('customf.pkl', 'rb'))
    prediction = model.predict(final_features)
    print("Prediction Done")
    df2['dep_delayed_15min'] = prediction
    df2['NAME'] = 'KRESHNAA'
    df2.rename(columns={'FlightDate': 'FLIGHTDATE'}, inplace=True)
    df2.rename(columns={'UniqueCarrier': 'UNIQUECARRIER'}, inplace=True)
    df2.rename(columns={'Origin': 'ORIGIN'}, inplace=True)
    df2.rename(columns={'Dest': 'DEST'}, inplace=True)
    df2.rename(columns={'Distance': 'DISTANCE'}, inplace=True)
    print('Snowflake Writeback Started')
    # Write the data from the DataFrame to the table named "FSDS_predictions".
    success, nchunks, nrows, _ = write_pandas(cnx, df2, 'FSDS_predictions')
    print('Snowflake Writeback Done')

In [14]:
tl.start()

while True:
  try:
    time.sleep(1)
  except KeyboardInterrupt:
    tl.stop()
    break

[2021-04-11 15:26:26,411] [timeloop] [INFO] Starting Timeloop..
[2021-04-11 15:26:26,411] [timeloop] [INFO] Starting Timeloop..
[2021-04-11 15:26:26,411] [timeloop] [INFO] Starting Timeloop..
[2021-04-11 15:26:26,411] [timeloop] [INFO] Starting Timeloop..
[2021-04-11 15:26:26,411] [timeloop] [INFO] Starting Timeloop..
[2021-04-11 15:26:26,411] [timeloop] [INFO] Starting Timeloop..
[2021-04-11 15:26:26,411] [timeloop] [INFO] Starting Timeloop..
[2021-04-11 15:26:26,423] [timeloop] [INFO] Registered job <function sf_wb at 0x7f9b660944c0>
[2021-04-11 15:26:26,423] [timeloop] [INFO] Registered job <function sf_wb at 0x7f9b660944c0>
[2021-04-11 15:26:26,423] [timeloop] [INFO] Registered job <function sf_wb at 0x7f9b660944c0>
[2021-04-11 15:26:26,423] [timeloop] [INFO] Registered job <function sf_wb at 0x7f9b660944c0>
[2021-04-11 15:26:26,423] [timeloop] [INFO] Registered job <function sf_wb at 0x7f9b660944c0>
[2021-04-11 15:26:26,423] [timeloop] [INFO] Registered job <function sf_wb at 0x7f

<ipython-input-9-76d21708a2bb>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['FlightDate'] = pd.to_datetime(X['FlightDate'],format='%Y-%m-%d')
<ipython-input-9-76d21708a2bb>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Day_of_Week'] = X['FlightDate'].dt.day_name()
<ipython-input-9-76d21708a2bb>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

features generated
Upload_Date           object
FlightDate            object
DEPTIME               object
UniqueCarrier         object
Origin                object
Dest                  object
Distance              object
dep_delayed_15min    float64
dtype: object
Prediction Done
Snowflake Writeback Started


<ipython-input-13-0856cf526884>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['dep_delayed_15min'] = prediction
<ipython-input-13-0856cf526884>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['NAME'] = 'KRESHNAA'
/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

Snowflake Writeback Done
[2021-04-11 17:50:23,428] [timeloop] [INFO] Stopping job <function sf_wb at 0x7f9b660944c0>
[2021-04-11 17:50:23,428] [timeloop] [INFO] Stopping job <function sf_wb at 0x7f9b660944c0>
[2021-04-11 17:50:23,428] [timeloop] [INFO] Stopping job <function sf_wb at 0x7f9b660944c0>
[2021-04-11 17:50:23,428] [timeloop] [INFO] Stopping job <function sf_wb at 0x7f9b660944c0>
[2021-04-11 17:50:23,428] [timeloop] [INFO] Stopping job <function sf_wb at 0x7f9b660944c0>
[2021-04-11 17:50:23,428] [timeloop] [INFO] Stopping job <function sf_wb at 0x7f9b660944c0>
[2021-04-11 17:50:23,428] [timeloop] [INFO] Stopping job <function sf_wb at 0x7f9b660944c0>
[2021-04-11 17:50:23,439] [timeloop] [INFO] Timeloop exited.
[2021-04-11 17:50:23,439] [timeloop] [INFO] Timeloop exited.
[2021-04-11 17:50:23,439] [timeloop] [INFO] Timeloop exited.
[2021-04-11 17:50:23,439] [timeloop] [INFO] Timeloop exited.
[2021-04-11 17:50:23,439] [timeloop] [INFO] Timeloop exited.
[2021-04-11 17:50:23,439] 